# NIST Uncertainty Example
This notebook is a set of examples for adding SNIST and Type Uncertainties to Files and Plotting them
Written by AWS 11/2016


In [2]:
# import 
import re
import sys
import os
from pyMeasure import *
from pyMeasure.Code.Analysis.NISTUncertainty import *
import numpy as np

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [5]:
# import a raw data file 
raw_one_port=OnePortRawModel(os.path.join(TESTS_DIRECTORY,'OnePortRawTestFile.txt'))
raw_one_port.show()

In [7]:
raw_one_port.metadata

{'Calibration_Date': '10 Aug 1998',
 'Calibration_Name': 'C980810.a1',
 'Connector_Type_Calibration': 'Type N Fem',
 'Connector_Type_Measurement': 'Type N',
 'Device_Description': 'n ck std open',
 'Device_Id': 'CTN112',
 'Measurement_Date': '8 Jun 2000',
 'Measurement_Time': '16:27:53',
 'Measurement_Type': '1-port',
 'Nbs': '1',
 'Number_Connects': '3',
 'Number_Frequencies': '28',
 'Number_Repeats': '1',
 'Operator': 'afm',
 'Port_Used': '1',
 'Program_Revision': '960910.143',
 'Program_Used': 'MEAS95',
 'Start_Frequency': '7',
 'System_Id': 'System I',
 'System_Letter': 'A'}

In [10]:
# now use the S_NIST function to create uncertainties
uncertainty_list=[]
uncertainty_mag=[]
uncertainty_phase=[]
for index,frequency in enumerate(raw_one_port.getColumn('Frequency')):
    row=raw_one_port.data[index]
    type_connector=raw_one_port.metadata["Connector_Type_Measurement"]
    arguments={'frequency':frequency,'parameter':'S11','magnitude':row[3],
               'wr_connector_type':type_connector}
    uncertainty=S_NIST(**arguments)
    uncertainty_mag.append(uncertainty[0])
    uncertainty_phase.append(uncertainty[1])
    uncertainty_list.append(uncertainty)


In [15]:
import matplotlib.pyplot as plt
frequency=raw_one_port.getColumn('Frequency')
direct_calc=[10.0**(-3.327+.046*f) for f in frequency]
plt.plot(frequency,uncertainty_mag)
plt.plot(frequency,direct_calc)
plt.show()


In [12]:
import matplotlib.pyplot as plt
plt.plot(raw_one_port.getColumn('Frequency'),uncertainty_phase)
plt.show()

In [21]:
# now use the S_NIST function to create uncertainties
b_uncertainty_list=[]
b_uncertainty_mag=[]
b_uncertainty_phase=[]

for index,frequency in enumerate(raw_one_port.getColumn('Frequency')):
    row=raw_one_port.data[index]
    type_connector=raw_one_port.metadata["Connector_Type_Measurement"]
    arguments={'frequency':frequency,'parameter':'S11','magnitude':row[3],
               'wr_connector_type':type_connector}
    uncertainty=type_b(**arguments)
    b_uncertainty_mag.append(uncertainty[0])
    b_uncertainty_phase.append(uncertainty[1])
    b_uncertainty_list.append(uncertainty)
frequency=raw_one_port.getColumn('Frequency')
plt.plot(frequency,b_uncertainty_mag)
plt.show()

In [20]:
len(frequency)

TypeError: object of type 'float' has no len()

In [23]:
print raw_one_port

#System I
#A
#Type N Fem
#Type N
#1-port
#8 Jun 2000
#16:27:53
#MEAS95
#960910.143
#afm
#C980810.a1
#10 Aug 1998
#1
#3
#1
#1
#28
#7
#n ck std open
#CTN112
Frequency,Direction,Connect,mag,arg
0.01000,1,1,0.9996,-0.47
0.01000,1,2,0.9996,-0.48
0.01000,1,3,0.9996,-0.48
0.02000,1,1,1.0001,-0.91
0.02000,1,2,1.0001,-0.92
0.02000,1,3,1.0001,-0.92
0.03000,1,1,0.9998,-1.35
0.03000,1,2,0.9998,-1.35
0.03000,1,3,0.9998,-1.35
0.04000,1,1,0.9997,-1.80
0.04000,1,2,0.9997,-1.80
0.04000,1,3,0.9997,-1.80
0.05000,1,1,0.9998,-2.25
0.05000,1,2,0.9998,-2.25
0.05000,1,3,0.9998,-2.25
0.06000,1,1,0.9999,-2.70
0.06000,1,2,0.9999,-2.70
0.06000,1,3,0.9999,-2.70
0.07000,1,1,1.0000,-3.15
0.07000,1,2,0.9999,-3.15
0.07000,1,3,0.9999,-3.15
0.08000,1,1,1.0000,-3.60
0.08000,1,2,1.0000,-3.60
0.08000,1,3,0.9999,-3.60
0.09000,1,1,1.0001,-4.04
0.09000,1,2,1.0000,-4.04
0.09000,1,3,1.0000,-4.04
0.10000,1,1,0.9998,-4.49


In [25]:
frequency=.08000
filter(lambda x:x[0]==frequency,raw_one_port.data)


[[0.08, 1, 1, 1.0, -3.6], [0.08, 1, 2, 1.0, -3.6], [0.08, 1, 3, 0.9999, -3.6]]

In [29]:
np.mean(np.array(filter(lambda x:x[0]==frequency,raw_one_port.data)),axis=0).tolist()

[0.08, 1.0, 2.0, 0.9999666666666668, -3.6]

In [30]:
np.std(np.array(filter(lambda x:x[0]==frequency,raw_one_port.data)),axis=0).tolist()

[0.0, 0.0, 0.816496580927726, 4.714045207909797e-05, 0.0]

In [33]:
unique_freq_list=sorted(list(set(raw_one_port.getColumn('Frequency'))))

In [34]:
unique_freq_list

[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

In [ ]:
def calrep_raw_data(raw_model):
    "Creates a callrep type model"
    unique_frequency_list=sorted(list(set(raw_model.getColumn('Frequency'))))
    mean_data=[]
    for frequency in unique_frequency_list:
        mean_row=np.mean(np.array(filter(lambda x:x[0]==frequency,raw_model.data)),axis=0).tolist()
        new_row=[frequency,mean_row]
        